In [36]:
# Import necessary modules

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [37]:
# pd.set_option('display.max_rows', None)

In [38]:
# Load Dataset

df = pd.read_csv('Bengaluru_House_Data.csv',encoding="ISO-8859-1" )
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,$39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,$120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,$62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,$95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,$51.00


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,$39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,$120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,$62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,$95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,$51.00


In [39]:
print(df.shape)
df.info()

(13320, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  object 
dtypes: float64(2), object(7)
memory usage: 936.7+ KB
(13320, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          1

# **Data Preprocessing**

### *Check Duplicates*

In [40]:
df.duplicated().sum()

511

511

In [41]:
df.drop_duplicates(inplace=True)

### *check nulls* -
##### (Handling missing values)

In [42]:
df.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5335
total_sqft         0
bath              73
balcony          605
price              0
dtype: int64

area_type          0
availability       0
location           1
size              16
society         5335
total_sqft         0
bath              73
balcony          605
price              0
dtype: int64

#####  society

In [43]:
df['society'].value_counts().head()

GrrvaGr    69
PrarePa    63
Sryalan    56
GMown E    54
Prtates    54
Name: society, dtype: int64

GrrvaGr    69
PrarePa    63
Sryalan    56
GMown E    54
Prtates    54
Name: society, dtype: int64

In [44]:
df['society'].nunique()

2688

2688

In [45]:
# This step may induce bias as there is more tha 40% data is missing in society column can dropping data is out of question.

df['society'].fillna('independent', inplace= True)
df['society'] = df['society'].str.lower()

##### Balcony

In [46]:
df['balcony'].value_counts()

2.0    4851
1.0    4712
3.0    1631
0.0    1010
Name: balcony, dtype: int64

2.0    4851
1.0    4712
3.0    1631
0.0    1010
Name: balcony, dtype: int64

In [47]:
# Assume missing values in balcony column indicates no balcony

df['balcony'].fillna(0.0, inplace= True)
df['balcony'] = df['balcony'].astype(int)

##### Location

In [48]:
# Dropping null values in location column as there is only 1 null value

df.dropna(subset=['location'], inplace=True)

##### Total_Sqft

In [49]:
df['total_sqft'].value_counts().tail()

2563 - 2733    1
2005           1
605 - 624      1
4260 - 4408    1
4689           1
Name: total_sqft, dtype: int64

2563 - 2733    1
2005           1
605 - 624      1
4260 - 4408    1
4689           1
Name: total_sqft, dtype: int64

In [50]:
# Feature engineering

def convert_total_sqft(sqft_value):
    if '-' in sqft_value:  # Check if there is a range
        start, end = map(float, sqft_value.split('-')) # Split the range and calculate the mean
        sqft_value = (start + end) / 2 
    elif 'Sq. Meter' in sqft_value:    # Convert square meter to square feet
        sqft_value = float(sqft_value.replace('Sq. Meter', '')) * 10.7639
    elif 'Sq. Yards' in sqft_value:  # Convert square yards to square feet
        sqft_value = float(sqft_value.replace('Sq. Yards', '')) * 9
    elif 'Acres' in sqft_value:  # Convert acres to square feet
        sqft_value = float(sqft_value.replace('Acres', '')) * 43560
    elif 'Guntha' in sqft_value:  # Convert Guntha to square feet
        sqft_value = float(sqft_value.replace('Guntha', '')) * 1089
    elif 'Cents' in sqft_value:  # Convert cents to square feet
        sqft_value = float(sqft_value.replace('Cents', '')) * 435.6
    elif 'Grounds' in sqft_value:  # Convert Grounds to square feet
        sqft_value = float(sqft_value.replace('Grounds', '')) * 2400
    elif 'Perch' in sqft_value:  # Convert Perch to square feet
        sqft_value = float(sqft_value.replace('Perch', '')) * 272.25
    elif sqft_value.isdigit():  # If no suffix, leave as is
        sqft_value = float(sqft_value)
    return sqft_value

In [51]:
# Apply the function to the total_sqft column
df['total_sqft'] = df['total_sqft'].apply(convert_total_sqft)
df['total_sqft'] = df['total_sqft'].astype(float)
# df['total_sqft'] = df['total_sqft'].round().astype(int)

##### size

In [52]:
result = df.groupby('size')['total_sqft'].agg(['max', 'min', lambda x: x.mode().iloc[0], 'mean'])
result.columns = ['Max', 'Min', 'Mode', 'Mean']
print(result)

                    Max         Min     Mode          Mean
size                                                      
1 BHK         10763.900     15.0000    500.0    693.453465
1 Bedroom    231303.600     45.0000    600.0   5162.029038
1 RK            905.000    296.0000    510.0    493.192308
10 BHK        12000.000   4000.0000   4000.0   8000.000000
10 Bedroom     7200.000    750.0000   1200.0   2646.333333
11 BHK         6000.000   5000.0000   5000.0   5500.000000
11 Bedroom     1200.000   1200.0000   1200.0   1200.000000
12 Bedroom     2232.000   2232.0000   2232.0   2232.000000
13 BHK         5425.000   5425.0000   5425.0   5425.000000
14 BHK         1250.000   1250.0000   1250.0   1250.000000
16 BHK        10000.000  10000.0000  10000.0  10000.000000
18 Bedroom     1200.000   1200.0000   1200.0   1200.000000
19 BHK         2000.000   2000.0000   2000.0   2000.000000
2 BHK          9900.000    302.0000   1200.0   1135.882210
2 Bedroom   1306800.000    276.0000   1200.0   5818.7182

In [53]:
# Function to fill missing values in size column

def fill_size_based_on_total_sqft(df):
    # Define conditions for size based on total_sqft ranges
    conditions = [
        (df['total_sqft'] >= 500) & (df['total_sqft'] <= 900),
        (df['total_sqft'] > 900) & (df['total_sqft'] <= 1400),
        (df['total_sqft'] > 1400) & (df['total_sqft'] <= 2500),
        (df['total_sqft'] > 2500) & (df['total_sqft'] <= 3200),
        (df['total_sqft'] > 3200)
    ]
    # Define corresponding size values
    sizes = ['1 BHK', '2 BHK', '3 BHK', '4 BHK', '5 BHK']

    # Fill null values in size column based on conditions
    df['size'] = np.where(df['size'].isnull(), np.select(conditions, sizes, default=df['size']), df['size'])
    

In [54]:
fill_size_based_on_total_sqft(df)

##### bath

In [55]:
df['bath'].value_counts()

2.0     6561
3.0     3180
4.0     1198
1.0      764
5.0      520
6.0      271
7.0      102
8.0       64
9.0       42
10.0      13
12.0       7
13.0       3
11.0       3
16.0       2
27.0       1
40.0       1
15.0       1
14.0       1
18.0       1
Name: bath, dtype: int64

2.0     6561
3.0     3180
4.0     1198
1.0      764
5.0      520
6.0      271
7.0      102
8.0       64
9.0       42
10.0      13
12.0       7
13.0       3
11.0       3
16.0       2
27.0       1
40.0       1
15.0       1
14.0       1
18.0       1
Name: bath, dtype: int64

In [56]:
result = df.groupby('bath')['total_sqft'].agg(['max', 'min', lambda x: x.mode().iloc[0], 'mean'])
result.columns = ['Max', 'Min', 'Mode', 'Mean']
result.head()

,Max,Min,Mode,Mean
bath,,,,
1.0,231303.600,15.0000,600.0,1331.925852
2.0,1306800.000,24.0000,1200.0,1475.191305
3.0,653400.000,11.0000,1200.0,1972.569412
4.0,36000.000,1.0000,1200.0,2509.057967
5.0,32722.256,258.3336,1200.0,2838.395134


,Max,Min,Mode,Mean
bath,,,,
1.0,231303.600,15.0000,600.0,1331.925852
2.0,1306800.000,24.0000,1200.0,1475.191305
3.0,653400.000,11.0000,1200.0,1972.569412
4.0,36000.000,1.0000,1200.0,2509.057967
5.0,32722.256,258.3336,1200.0,2838.395134


In [57]:
# Function to fill missing values in bath column

def fill_bath_based_on_total_sqft(df):
    conditions = [
        (df['total_sqft'] >= 500) & (df['total_sqft'] <= 1200),
        (df['total_sqft'] > 1200) & (df['total_sqft'] <= 1400),
        (df['total_sqft'] > 1400) & (df['total_sqft'] <= 2000),
        (df['total_sqft'] > 2000) & (df['total_sqft'] <= 2800),
        (df['total_sqft'] > 2800)
    ]
    bath = [1, 2, 3, 4, 5]
    df['bath'] = np.where(df['bath'].isnull(), 
                np.select(conditions, bath, default=df['bath']), df['bath'])
    

In [58]:
fill_bath_based_on_total_sqft(df)
df['bath']=df['bath'].astype(int)

- Now data has been cleaned of null values.

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12808 entries, 0 to 13318
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     12808 non-null  object 
 1   availability  12808 non-null  object 
 2   location      12808 non-null  object 
 3   size          12808 non-null  object 
 4   society       12808 non-null  object 
 5   total_sqft    12808 non-null  float64
 6   bath          12808 non-null  int32  
 7   balcony       12808 non-null  int32  
 8   price         12808 non-null  object 
dtypes: float64(1), int32(2), object(6)
memory usage: 900.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12808 entries, 0 to 13318
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     12808 non-null  object 
 1   availability  12808 non-null  object 
 2   location      12808 non-null  object 
 3   size          12808 non-nul

##### Price

In [60]:
df['price'].value_counts()

75       293
65       284
55       260
60       255
45       228
        ... 
65.3       1
97.78      1
51.19      1
39.97      1
488        1
Name: price, Length: 2205, dtype: int64

75       293
65       284
55       260
60       255
45       228
        ... 
65.3       1
97.78      1
51.19      1
39.97      1
488        1
Name: price, Length: 2205, dtype: int64

In [61]:
# Function to clean price column of punctuation and calc. mean if there's a range

def clean_price(df):
    df["price"] = df["price"].str.replace(r"[^\d\-+\.]", "", regex=True).apply(lambda x: float(x.split('-')[0]) if '-' not in x else (float(x.split('-')[0]) + float(x.split('-')[1])) / 2).round(2)
    return df["price"]

In [62]:
clean_price(df)

0         39.07
1        120.00
2         62.00
3         95.00
4         51.00
          ...  
13314    112.00
13315    231.00
13316    400.00
13317     60.00
13318    488.00
Name: price, Length: 12808, dtype: float64

0         39.07
1        120.00
2         62.00
3         95.00
4         51.00
          ...  
13314    112.00
13315    231.00
13316    400.00
13317     60.00
13318    488.00
Name: price, Length: 12808, dtype: float64

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12808 entries, 0 to 13318
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     12808 non-null  object 
 1   availability  12808 non-null  object 
 2   location      12808 non-null  object 
 3   size          12808 non-null  object 
 4   society       12808 non-null  object 
 5   total_sqft    12808 non-null  float64
 6   bath          12808 non-null  int32  
 7   balcony       12808 non-null  int32  
 8   price         12808 non-null  float64
dtypes: float64(2), int32(2), object(5)
memory usage: 900.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12808 entries, 0 to 13318
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     12808 non-null  object 
 1   availability  12808 non-null  object 
 2   location      12808 non-null  object 
 3   size          12808 non-nul

In [64]:
df['availability'].value_counts()

Ready To Move    10185
18-May             292
18-Dec             285
18-Apr             269
18-Aug             188
                 ...  
15-Aug               1
17-Jan               1
16-Nov               1
16-Jan               1
14-Jul               1
Name: availability, Length: 81, dtype: int64

Ready To Move    10185
18-May             292
18-Dec             285
18-Apr             269
18-Aug             188
                 ...  
15-Aug               1
17-Jan               1
16-Nov               1
16-Jan               1
14-Jul               1
Name: availability, Length: 81, dtype: int64

In [65]:
df['availability'] = df['availability'].replace('Immediate Possession', 'Ready To Move')

In [66]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,coomee,1056.0,2,1,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,theanmp,2600.0,5,3,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,independent,1440.0,2,3,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,soiewre,1521.0,3,1,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,independent,1200.0,2,1,51.00


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,coomee,1056.0,2,1,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,theanmp,2600.0,5,3,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,independent,1440.0,2,3,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,soiewre,1521.0,3,1,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,independent,1200.0,2,1,51.00


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12808 entries, 0 to 13318
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     12808 non-null  object 
 1   availability  12808 non-null  object 
 2   location      12808 non-null  object 
 3   size          12808 non-null  object 
 4   society       12808 non-null  object 
 5   total_sqft    12808 non-null  float64
 6   bath          12808 non-null  int32  
 7   balcony       12808 non-null  int32  
 8   price         12808 non-null  float64
dtypes: float64(2), int32(2), object(5)
memory usage: 900.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12808 entries, 0 to 13318
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     12808 non-null  object 
 1   availability  12808 non-null  object 
 2   location      12808 non-null  object 
 3   size          12808 non-nul

In [68]:
# Extract numeric part from 'size' column
df['house_size'] = df['size'].str.extract('(\d+)').astype(int)

# Handle 'BHK', 'Bedroom', and 'RK'
df['type'] = df['size'].apply(lambda x: 'BHK' if 'BHK' in x else ('Bedroom' if 'Bedroom' in x else 'RK'))

# Map type to numerical values
type_mapping = {'BHK': 0,'Bedroom': 1, 'RK': 2}
df['house_type'] = df['type'].map(type_mapping)

# Drop unnecessary columns
df.drop(['size','type'], axis=1, inplace=True)
df.head()

,area_type,availability,location,society,total_sqft,bath,balcony,price,house_size,house_type
0,Super built-up Area,19-Dec,Electronic City Phase II,coomee,1056.0,2,1,39.07,2,0
1,Plot Area,Ready To Move,Chikka Tirupathi,theanmp,2600.0,5,3,120.00,4,1
2,Built-up Area,Ready To Move,Uttarahalli,independent,1440.0,2,3,62.00,3,0
3,Super built-up Area,Ready To Move,Lingadheeranahalli,soiewre,1521.0,3,1,95.00,3,0
4,Super built-up Area,Ready To Move,Kothanur,independent,1200.0,2,1,51.00,2,0


,area_type,availability,location,society,total_sqft,bath,balcony,price,house_size,house_type
0,Super built-up Area,19-Dec,Electronic City Phase II,coomee,1056.0,2,1,39.07,2,0
1,Plot Area,Ready To Move,Chikka Tirupathi,theanmp,2600.0,5,3,120.00,4,1
2,Built-up Area,Ready To Move,Uttarahalli,independent,1440.0,2,3,62.00,3,0
3,Super built-up Area,Ready To Move,Lingadheeranahalli,soiewre,1521.0,3,1,95.00,3,0
4,Super built-up Area,Ready To Move,Kothanur,independent,1200.0,2,1,51.00,2,0


In [69]:
df.to_csv('cleaned_data.csv', index=False)

In [70]:
df.shape

(12808, 10)

(12808, 10)